In [2]:
from google.colab import drive 
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


# 장르 속성을 이용한 영화 콘텐츠 기반 필터링

콘텐츠 기반 필터링은 사용자가 특정 영화를 감상하고 그 영화를 좋아했따면 그 영화와 비슷한 특성/ 속성, 구성요소 등을 가진 다른 영화를 추천하는 것.

만약, 영화 인셉션을 재밌게 봤다면 인셉션의 장르인 액션, 공상과학으로 높은 평점을 받은 다른 영화를 추천하거나 인셉션의 감독인 크리스토퍼 놀란의 다른 영화를 추천하는 방식.

** 영화 장르 속성을 기반으로 구성**

In [3]:
import pandas as pd 
import numpy as np
import warnings; warnings.filterwarnings('ignore')

In [4]:
movies = pd.read_csv('/content/gdrive/My Drive/data/movie/tmdb_5000_movies.csv')
print(movies.shape)
movies.head(1)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


tmdb_5000_movies.csv 는 4803개의 레코드와 20개의 피처로 구성되어있습니다. 이 중 콘텐츠 기반 필터링 추천 분석에 사용할 주요 컬럼만 추출해 새롭게 DataFrame으로 만들겠습니다.

추출할 주요 컬럼은 id, 영화제목 title, 영화가 속한 genres, 평균 평점인 vote_averge, 평점 투표 수인 vote_count, 영화의 인기를 나타내는 popularity, 영화를 설명하는 주요 키워드 문구인 keywords, 영화에 대한 개요 설명인 overview 입니다.

In [6]:
movies_df = movies[['id','title','genres','vote_average','vote_count','popularity','keywords','overview']]

여기서 주의해야할 점은, 'genres', 'keywords'와 같은 컬럼에는 파이썬 리스트 내부에 여러 개의 딕셔너리가 있는 형태의 문자열로 표기되어있습니다. 

In [7]:
pd.set_option('max_colwidth',100)
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


genres 칼럼은 여러 개의 개별 장르 데이터를 갖고 있고, 이 개별 장르의 명칭은 딕셔너리의 key인 name으로 추출할 수 있습니다. keywords 또한 마찬가지입니다. 따라서 이 문자열들을 분해해, 파이썬 리스트 객체로 추출합니다.

In [8]:
from ast import literal_eval
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

In [11]:
movies_df['genres'] = movies_df['genres'].apply(lambda x : [y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [y['name'] for y in x])
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


# 장르 콘텐츠 유사도 측정

장르 칼럼을 기반으로 하는 콘텐츠 기반 필터링은 다음과 같은 단계로 구현하겠습니다.

1. 문자열로 변환된 genres 칼럼을 Count 기반으로 피쳐 벡터화 변환합니다.
2. genres 문자열을 피쳐벡터화 행렬로 변환한 데이터 세트를 코사인 유사도를 통해 비교합니다. 이를 위해 데이터 세트의 레코드 별로 타 레코드와 장르에서 코사인 유사도 값을 가지는 객체를 생성합니다.
3. 장르 유사도가 높은 영화 중에 평점이 높은 순으로 영화를 추천합니다.

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환
movies_df['genres_literal']=movies_df['genres'].apply(lambda x:(' ').join(x))
movies_df[['genres_literal']][:1]

,genres_literal
0,Action Adventure Fantasy Science Fiction


In [14]:
count_vect = CountVectorizer(min_df = 0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


In [15]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)

코사인 유사도를 통해 생성된 genre_sim 객체는 movies_df의 genre_literal 칼럼을 피처 벡터화한 행렬 데이터의 행별 유사도 정보를 가지고 있습니다.  
결국은 movies_df 데이터프레임의 행별 장르 유사도 값을 가지고 있는 것입니다.  

movies_df를 장르 기준으로 콘텐츠 기반 필터링을 수행하려면 movies_df의 개별 레코드에 대해 가장 장르 유사도가 높은 순으로 다른 레코드를 추출해야합니다.

genre_sim 객체의 기준 행별로 비교 대상이 되는 행의 유사도 값이 높은 순으로 정렬된 행의 위치 인덱스 값을 추출하면 이를 해결할 수 있습니다. 

In [16]:
genre_sim_sorted_ind = genre_sim.argsort()[:,::-1] #유사도가 높은 순으로 정리된 genre_sim 객체의 비교 행 위치 값을 얻을 수 있습니다.
print(genre_sim_sorted_ind[:1])

[[   0 3494  813 ... 3038 3037 2401]]


# 장르 콘텐츠 필터링을 이용한 영화 추천

이제 장르 유사도에 따라 영화를 추천하는 함수를 생성합니다.
고객이 선정한 추천 기준이 되는 영화 제목, 추천할 영화 건수를 입력하면 추천 영화 정보를 가지는 DataFrame을 반환합니다.

In [17]:
def find_sim_movie(df ,sorted_ind, title_name, top_n = 10):
  # 인자로 입력된 df에서 'title' 칼럼이 입력된 title_name 값인 데이터프레임 추출
  title_movie = df[df['title'] == title_name]

  title_index = title_movie.index.values
  similar_indexes = sorted_ind[title_index, :(top_n)]

  print(similar_indexes)
  similar_indexes = similar_indexes.reshape(-1) #1차원 array로 변경

  return df.iloc[similar_indexes]

In [20]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title','vote_average']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


The Godfather(대부) 라는 영화를 입력했을 때, 대부 2편이 가장 먼저 추천되었고 이 외에도 대부를 재미있게 봤다면 즐길 비슷한 유형의 영화들이 추천되었습니다.  
하지만 낯선 영화도 많습니다. 평점이 0.0인 영화도 포함되어있습니다. 이는 개선이 필요합니다.

이번에는 더 많은 후보군을 선정한 뒤에 영화의 평점에 따라 필터링해서 최종 추천하는 방식으로 변경하겠습니다. 영화의 평점 정보인 `vote_average` 값을 이용해봅시다. 영화 평점은 좋은 요소가 되지만, 만약 1,2명과 같은 소수의 관객이 높은 점수를 준 경우에는 왜곡된 값을 가질 수 있습니다.

In [21]:
movies_df[['title','vote_average','vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


위에서 확인할 수 있듯이, 총 1명이 10점을 준 경우와, 총 8205명의 평점 평균으로 8.5점을 받게 된 경우가 존재하는 것을 확인할 수 있습니다.  
우리는 가중 평점을 활용하여 새로운 평가 방식을 도입해야합니다.

가중 평점 = (v/(v+m)) * R + (m/(v+m)) \* C

* v: 개별 영화에 평점을 투표한 횟수
* m: 평점을 부여하기 위한 최소 투표 횟수
* R: 개별 영화에 대한 평균 평점
* C: 전체 영화에 대한 평균 평점

v 는 'vote_count' 값이며, R은 'vote_average' 값에 해당합니다. C의 경우 전체 영화의 평균 평점이므로 movies_df['vote_average'].mean()으로 구할 수 있습니다.  
m 의 경우는 투표 횟수에 따른 가중치를 직접 조절하는 역할을 하는데, m을 높이면 평점 투표 횟수가 많은 영화에 더 많은 가중 평점을 주게 됩니다.  
여기서는 m을 상위 60% 해당하는 횟수를 기준으로 정하겠습니다.

In [22]:
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)
print('C:',round(C,3),'m:',round(m,3))

C: 6.092 m: 370.2


In [23]:
percentile = 0.6
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)

def weighted_vote_average(record):
  v = record['vote_count']
  R = record['vote_average']

  return ((v/(v+m)) * R) +( (m/(v+m)) * C)

movies['weighted_vote'] = movies.apply(weighted_vote_average, axis = 1)

In [25]:
movies_df = movies[['id','title','genres','vote_average','weighted_vote','vote_count','popularity','keywords','overview']]

In [26]:
movies_df[['title','vote_average','weighted_vote','vote_count']].sort_values('weighted_vote', ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


이제, 새롭게 정의된 평점 기준에 따라 영화를 추천해보겠습니다.  
장르 유사성이 높은 영화를 top_n의 2배수만큼 후보군으로 선정한 뒤, weighted_vote 값이 높은 순으로 top_n개를 추출하는 방식을 이용해봅시다.

In [27]:
def find_sim_movie(df, sorted_ind, title_name, top_n = 10):
    # 인자로 입력된 df에서 'title' 칼럼이 입력된 title_name 값인 데이터프레임 추출
    title_movie = df[df['title'] == title_name]

    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    similar_indexes = similar_indexes.reshape(-1) #1차원 array로 변경

    similar_indexes = similar_indexes[similar_indexes != title_index]

    return df.iloc[similar_indexes].sort_values('weighted_vote',ascending=False)[:top_n]

similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title','vote_average','weighted_vote']]

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427


앞선 추천보다 더 나은 추천 목록을 제시하는 것을 확인할 수 있습니다.  
하지만 장르만으로는 영화가 전달하는 많은 요소와 분위기, 그리고 개인이 좋아하는 성향을 반영하기에는 부족할 수 있습니다. 아마 좋아하는 영화배우나 감독을 보고 영화를 선택하는 경우가 더 많을 수 있습니다.